In [1]:
from preset import set_proxy_and_token
set_proxy_and_token()

In [ ]:
import torch
from diffusers import DiffusionPipeline

base_model = "black-forest-labs/FLUX.1-dev"
pipe = DiffusionPipeline.from_pretrained(
    base_model, 
    torch_dtype=torch.bfloat16
)

lora_repo = "strangerzonehf/Flux-Midjourney-Mix2-LoRA"
trigger_word = "MJ v6"  
pipe.load_lora_weights(lora_repo)

pipe.enable_sequential_cpu_offload()

# device = torch.device("cuda")
# pipe.to(device)

In [ ]:
prompt = "cinematic photo of Godzilla eating sushi with a cat in a izakaya, 35mm photograph, film, professional, 4k, highly detailed"
generator = torch.Generator(device="cuda").manual_seed(1)
image = pipe(prompt, generator=generator).images[0]
image

In [ ]:
f

In [2]:
import torch
from diffusers import AutoPipelineForText2Image
torch._inductor.config.conv_1x1_as_mm = True
torch._inductor.config.coordinate_descent_tuning = True
torch._inductor.config.epilogue_fusion = False
torch._inductor.config.coordinate_descent_check_all_directions = True

pipe = AutoPipelineForText2Image.from_pretrained(
    pretrained_model_or_path="stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.bfloat16
)

pipe.unet.to(memory_format=torch.channels_last)
pipe.vae.to(memory_format=torch.channels_last)
pipe.unet = torch.compile(pipe.unet, mode="max-autotune", fullgraph=True)
pipe.vae.decode = torch.compile(pipe.vae.decode, mode="max-autotune", fullgraph=True)
pipe.enable_model_cpu_offload()


image = pipe(
    prompt="An astronaut riding a green horse",
    generator=torch.Generator(device="cuda").manual_seed(3),

).images[0]
    
image.save("astronaut2.png")

2025-02-04 12:18:45.554660: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 12:18:45.576076: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738642725.602483  145794 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738642725.610622  145794 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 12:18:45.642725: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

W0204 12:20:32.844000 145794 site-packages/torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode
AUTOTUNE addmm(8192x640, 8192x320, 320x640)
  bias_addmm 0.1249 ms 100.0% 
  addmm 0.1413 ms 88.4% 
SingleProcess AUTOTUNE benchmarking takes 0.2415 seconds and 0.0004 seconds precompiling for 2 choices
AUTOTUNE addmm(2048x1280, 2048x640, 640x1280)
  bias_addmm 0.1014 ms 100.0% 
  addmm 0.1239 ms 81.8% 
SingleProcess AUTOTUNE benchmarking takes 0.2235 seconds and 0.0004 seconds precompiling for 2 choices
AUTOTUNE addmm(2048x1280, 2048x2560, 2560x1280)
  bias_addmm 0.3533 ms 100.0% 
  addmm 0.3707 ms 95.3% 
SingleProcess AUTOTUNE benchmarking takes 0.2360 seconds and 0.0004 seconds precompiling for 2 choices
AUTOTUNE addmm(2048x1280, 2048x1920, 1920x1280)
  bias_addmm 0.2703 ms 100.0% 
  addmm 0.2877 ms 94.0% 
SingleProcess AUTOTUNE benchmarking takes 0.2311 seconds and 0.0005 seconds precompiling for 2 choices
AUTOTUNE addmm(8192x640, 8192x1920, 1920x640)
  bias_

Unsupported: torch.* op returned non-Tensor device call_function <built-in function getitem>

from user code:
   File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/diffusers/utils/accelerate_utils.py", line 45, in wrapper
    self._hf_hook.pre_forward(self)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 706, in pre_forward
    self.prev_module_hook.offload()
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 723, in offload
    self.hook.init_hook(self.model)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 702, in init_hook
    return module.to("cpu")
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/diffusers/models/modeling_utils.py", line 1077, in to
    return super().to(*args, **kwargs)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/torch

In [3]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [4]:
image = pipe(
    prompt="An astronaut riding a green horse",
    generator=torch.Generator(device="cuda").manual_seed(3),

).images[0]
    
image.save("astronaut2.png")

  0%|          | 0/50 [00:00<?, ?it/s]

Unsupported: torch.* op returned non-Tensor device call_function <built-in function getitem>

from user code:
   File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/diffusers/utils/accelerate_utils.py", line 45, in wrapper
    self._hf_hook.pre_forward(self)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 706, in pre_forward
    self.prev_module_hook.offload()
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 723, in offload
    self.hook.init_hook(self.model)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/accelerate/hooks.py", line 702, in init_hook
    return module.to("cpu")
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/diffusers/models/modeling_utils.py", line 1077, in to
    return super().to(*args, **kwargs)
  File "/home/xinchengo/miniconda3/envs/seminar/lib/python3.12/site-packages/torch